# Time Zone
## Bot to tell <span style="color:red">Current Time</span> for a given <span style="color:red">Location</span>. ## 

In [1]:
import pickle
from geopy import geocoders
from datetime import datetime
from itertools import groupby
from ipywidgets import widgets  
from pycorenlp import StanfordCoreNLP

In [2]:
# config
# get your Google Maps API Key from google consol
api_key = "<GOOGLE_API_KEY>" 
geo = geocoders.GoogleV3(api_key=api_key['google'])

# Stanford CoreNLP server url
URL = 'http://localhost:9000/'
nlp = StanfordCoreNLP(URL)

In [3]:
test_text = """
what is the time now?
what is the time in tokyo?
what is the time in rio de janeiro?
what is the time in Chennai, Tamil Nadu?
""".strip().split('\n')

### Tag Location Entity in the Sentence
- Stanford CoreNLP server will process the text and return POS tagged tuples

In [4]:
# true case & ner
def named_entity_tagging(sent):
    properties = { 'annotators': 'tokenize,ssplit,pos,lemma,truecase,ner',
                   'outputFormat': 'json'}
    
    response_ner = nlp.annotate(sent.title(), properties=properties)
    
    return [(dic.get('truecaseText'), dic.get('ner')) for dic in response_ner['sentences'][0]['tokens']]

In [5]:
named_entity_tagging("time now in rio de jenirio")

[('Time', 'O'),
 ('now', 'DATE'),
 ('in', 'O'),
 ('Rio', 'LOCATION'),
 ('de', 'LOCATION'),
 ('Jenirio', 'LOCATION')]

### Extract Location Entity 
- Extract only the location entity
- If the location entity is made up of multiple words then chunk the entity

In [6]:
def extract_location(sent, entity_label='LOCATION', default='Chennai'):
    named_entity_lst = named_entity_tagging(sent)
    entity_lst = list()
    for tag, grp in groupby(named_entity_lst, lambda entity:entity[1]):
        if tag == entity_label:
            entity_lst.append(" ".join(entity for entity,_ in grp))
    if not entity_lst:
        entity_lst.append(default)
    return entity_lst

In [7]:
extract_location("time now in rio de jenirio")

['Rio de Jenirio']

### Fine Timezone Corresponding to the Location
- Get the geocode for the location from Google Geocoding API
- With Geocode find timezone with Google Timezone API

In [8]:
def location_timezone(locations):
    tz = list()
    for location in locations:
        timezone = geo.timezone(geo.geocode(location).point)
        tz.append(timezone)
    if len(set(tz)) == 1:
        return tz[0]
    else:
        return zip(locations, tz)        

In [11]:
location_timezone(['Rio de Jenirio'])

<DstTzInfo 'America/Sao_Paulo' LMT-1 day, 20:54:00 STD>

In [12]:
def bot(sent):
    location = extract_location(sent)
    tz_info = location_timezone(location)
    return datetime.now(tz_info).strftime('%a, %d %b %Y %I:%M:%S %p %Z')

In [13]:
bot("time now in rio de jenirio")

'Sat, 22 Jul 2017 08:55:18 AM BRT'

In [ ]:
text = widgets.Text()
display(text)

def submit(data):
    print("{}\n{}".format(data.value, bot(data.value)))
    
text.on_submit(submit)